# Preprocessing Pipeline
## Import Statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import sklearn
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Functions

## Reading in the data

In [2]:
cpi_index = pd.read_csv('data/cpi.csv')
cpi_index.head()

,Quarter,CPI
0,2021Q1,117.9
1,2021Q2,118.8
2,2021Q3,119.7
3,2021Q4,121.3
4,2022Q1,123.9


In [3]:
df = pd.read_csv('data/data.csv', encoding = 'windows-1252')
df.head()

/tmp/ipykernel_33858/2365483159.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/data.csv', encoding = 'windows-1252')


,FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,IsDeleted,PriceIncGSTRaw,VYMM,VMakeModel,VMake,VYear,VMid,VRego,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass
0,2,1449822,2024-02-19,33. Approved,2024-02-19 16:01:15,64. Completed,2323471,Log,"Logbook Service - 75,000 km / 60 months",0,0,False,464.27,2019 AUDI A5 COUPE,AUDI A5 COUPE,AUDI,2019.0,AUD45780,YPG67B,17177,2606,ACT,Canberra,2.0
1,2,1565721,2024-03-19,33. Approved,2024-03-19 13:56:15,64. Completed,2537851,Capped,"Capped Price Service - 10,000 km",0,0,False,290.00,2022 TOYOTA HILUX,TOYOTA HILUX,TOYOTA,2022.0,TOY47062,1HSB467,1133,6017,WA,Perth,2.0
2,1,1320451,2023-11-13,33. Approved,2023-11-14 13:55:17,64. Completed,2053300,ProductTyreOp,((Products)),0,0,False,60.50,2001 NISSAN PATROL,NISSAN PATROL,NISSAN,2001.0,NIS29550,DE18YN,14790,2263,NSW,Central Coast,3.0
3,1,1320451,2023-11-13,33. Approved,2023-11-14 13:55:17,64. Completed,2053301,ProductTyre,((Products)),0,0,False,1484.00,2001 NISSAN PATROL,NISSAN PATROL,NISSAN,2001.0,NIS29550,DE18YN,14790,2263,NSW,Central Coast,3.0
4,2,1448252,2024-02-19,33. Approved,2024-02-19 11:30:01,64. Completed,2320054,Capped,"Capped Price Service - 105,000 km",0,0,False,358.00,2019 HYUNDAI SANTA FE,HYUNDAI SANTA FE,HYUNDAI,2019.0,HYU45824,1RE6XD,17962,3844,VIC,Traralgon,2.0


## Remove Rows

In [4]:
df = df[df['IsCustomService'] != '1']
df = df[df['IsCustomRepair'] != '1']
df = df[df['TaskName'] != '((Products))']
df = df[df['TaskName'] != '((Tyres))']
df = df[df['TaskName'] != 'Custom Repair']
df = df[df['TaskName'] != 'Tyre Replacement']
df = df[df['BTicketType'] != 'OtherTicket']
df = df[df['BTicketType'] != 'Custom']
df = df[df['BTicketType'] != 'Basic']

df = df[df['PriceIncGSTRaw'] > 0]
df.head()

,FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,IsDeleted,PriceIncGSTRaw,VYMM,VMakeModel,VMake,VYear,VMid,VRego,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass
0,2,1449822,2024-02-19,33. Approved,2024-02-19 16:01:15,64. Completed,2323471,Log,"Logbook Service - 75,000 km / 60 months",0,0,False,464.27,2019 AUDI A5 COUPE,AUDI A5 COUPE,AUDI,2019.0,AUD45780,YPG67B,17177,2606,ACT,Canberra,2.0
1,2,1565721,2024-03-19,33. Approved,2024-03-19 13:56:15,64. Completed,2537851,Capped,"Capped Price Service - 10,000 km",0,0,False,290.00,2022 TOYOTA HILUX,TOYOTA HILUX,TOYOTA,2022.0,TOY47062,1HSB467,1133,6017,WA,Perth,2.0
4,2,1448252,2024-02-19,33. Approved,2024-02-19 11:30:01,64. Completed,2320054,Capped,"Capped Price Service - 105,000 km",0,0,False,358.00,2019 HYUNDAI SANTA FE,HYUNDAI SANTA FE,HYUNDAI,2019.0,HYU45824,1RE6XD,17962,3844,VIC,Traralgon,2.0
5,2,1320517,2023-11-13,33. Approved,2023-11-13 11:36:59,64. Completed,2051287,Capped,"Capped Price Service - 75,000 km",0,0,False,230.00,2021 TOYOTA RAV4,TOYOTA RAV4,TOYOTA,2021.0,TOY45523,EQF85F,17262,2250,NSW,Central Coast,3.0
8,2,1551817,2024-03-15,16. Requires Changes,2024-03-15 11:53:03,64. Completed,2512709,Repair,Roadworthy Inspection / Pink Slips,0,0,False,46.00,2015 HYUNDAI I30,HYUNDAI I30,HYUNDAI,2015.0,HYU38277,DDR43Q,17596,2303,NSW,Newcastle,2.0


## Grouping Similar entries

In [5]:
df['Distance'] = None
df['Months'] = None

mask = df['BTicketType'] == 'Log'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Rest'] = split_1[1].fillna('')

split_2 = df.loc[mask, 'Rest'].str.split(' / ', n=1, expand=True)
df.loc[mask, 'Distance'] = split_2[0]
df.loc[mask, 'Months'] = split_2[1]

mask = df['BTicketType'] == 'Capped'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

mask = df['BTicketType'] == 'Prescribed'
split_1 = df.loc[mask, 'TaskName'].str.split(' - ', n=1, expand=True)
df.loc[mask, 'TaskName'] = split_1[0]
df.loc[mask, 'Distance'] = split_1[1].fillna('')

# Step 3: Drop the temporary column
df = df.drop(columns=['Rest'])

In [6]:
df['Months'] = df['Months'].astype(str).str.extract(r'(\d+)')
df['Months'] = pd.to_numeric(df['Months'], errors='coerce')

df['Distance'] = df['Distance'].astype(str).str.replace(',', '', regex=False)  
df['Distance'] = df['Distance'].astype(str).str.extract(r'(\d+)')
df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')

In [7]:

df.head()

,FCID,BookingID,BCreatedDateAEST,BStatusAfterSubmitted,BStatusFromDateTimeAEST,BStatusFinal,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,IsDeleted,PriceIncGSTRaw,VYMM,VMakeModel,VMake,VYear,VMid,VRego,BShopID,BShopPostcode,BShopState,BShopRegionName,BShopRegionClass,Distance,Months
0,2,1449822,2024-02-19,33. Approved,2024-02-19 16:01:15,64. Completed,2323471,Log,Logbook Service,0,0,False,464.27,2019 AUDI A5 COUPE,AUDI A5 COUPE,AUDI,2019.0,AUD45780,YPG67B,17177,2606,ACT,Canberra,2.0,75000.0,60.0
1,2,1565721,2024-03-19,33. Approved,2024-03-19 13:56:15,64. Completed,2537851,Capped,Capped Price Service,0,0,False,290.00,2022 TOYOTA HILUX,TOYOTA HILUX,TOYOTA,2022.0,TOY47062,1HSB467,1133,6017,WA,Perth,2.0,10000.0,NaN
4,2,1448252,2024-02-19,33. Approved,2024-02-19 11:30:01,64. Completed,2320054,Capped,Capped Price Service,0,0,False,358.00,2019 HYUNDAI SANTA FE,HYUNDAI SANTA FE,HYUNDAI,2019.0,HYU45824,1RE6XD,17962,3844,VIC,Traralgon,2.0,105000.0,NaN
5,2,1320517,2023-11-13,33. Approved,2023-11-13 11:36:59,64. Completed,2051287,Capped,Capped Price Service,0,0,False,230.00,2021 TOYOTA RAV4,TOYOTA RAV4,TOYOTA,2021.0,TOY45523,EQF85F,17262,2250,NSW,Central Coast,3.0,75000.0,NaN
8,2,1551817,2024-03-15,16. Requires Changes,2024-03-15 11:53:03,64. Completed,2512709,Repair,Roadworthy Inspection / Pink Slips,0,0,False,46.00,2015 HYUNDAI I30,HYUNDAI I30,HYUNDAI,2015.0,HYU38277,DDR43Q,17596,2303,NSW,Newcastle,2.0,NaN,NaN


## Grouping Rare Tasks

In [8]:
task_counts = df['TaskName'].value_counts()

# Define a threshold 
threshold = 50
rare_tasks = task_counts[task_counts < threshold].index

# Replace rare tasknames with 'Other'
print(len(df['TaskName'].unique()))
#df['TaskName'] = df['TaskName'].replace(rare_tasks, 'Other')
df = df[~df['TaskName'].isin(rare_tasks)].copy()
print(len(df['TaskName'].unique()))

1561
651


In [9]:

df_temp = df[df['BTicketType'].str.contains('repair', case=False)]
result = df_temp.groupby('TaskName').agg(
    Count=('TaskName', 'count'),
    AvgPrice=('PriceIncGSTRaw', 'mean')
).sort_values(by='Count', ascending=False)

print(result)


                                                    Count      AvgPrice
TaskName                                                               
Replace Wiper Blades                                51927     85.267843
Battery Replacement                                 28028    430.784536
Front Wheel Alignment                               25431     79.375187
Roadworthy Inspection / Pink Slips                  19169    131.705779
Front Brake Pad And Rotor Replacement               13872    871.085698
Cabin Filter Replacement                            12927     86.440590
Brake System Flush                                  12892     92.951233
Tyre Rotation Service                               12865     34.663377
Air Filter Replacement                              12256     96.386521
Front Brake Pad Replacement                         11740    393.242570
Front Wiper Inserts Replacement                     10040     76.924923
Rear Brake Pad Replacement                           9916    380

In [10]:
model_counts = df['VMakeModel'].value_counts()


threshold = 20
rare_models = model_counts[model_counts < threshold].index

# Replace rare tasknames with 'Other'
#df['TaskName'] = df['TaskName'].replace(rare_tasks, 'Other')
print(len(df['VMakeModel'].unique()))
df = df[~df['VMakeModel'].isin(rare_models)].copy()
print(len(df['VMakeModel'].unique()))

2004
632


## Feature Engineering
1. Create Date Column
2. Create Adjusted Price Column

In [11]:
# 1. Create date column
df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST'],  dayfirst=True);
df['Date'] = df['BCreatedDateAEST'].dt.date

/tmp/ipykernel_33858/224034733.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['BCreatedDateAEST'] = pd.to_datetime(df['BCreatedDateAEST'],  dayfirst=True);


In [12]:
# 2. Price Adjustment
df = df[df['PriceIncGSTRaw'] != 0]
df['Date'] = pd.to_datetime(df['Date']) # ensure 'date' is in datetime format
df['Quarter'] = df['Date'].dt.to_period('Q').astype(str)
df = df.merge(cpi_index, on='Quarter', how='left')
base_cpi = cpi_index['CPI'].iloc[-1]
df['AdjustedPrice'] = round(df['PriceIncGSTRaw'] * (base_cpi / df['CPI']), 2) # round to 2 decimal places for consistency
df['AdjustedPrice'] = round(df['AdjustedPrice'].fillna(df['PriceIncGSTRaw'])) # use original price where CPI is missing (for current quarter which does not need adjustment)


In [13]:
print("Quarters in df:", df['Quarter'].unique())
print("Quarters in cpi_index:", cpi_index['Quarter'].unique())

Quarters in df: ['2024Q1' '2023Q4' '2023Q3' '2023Q2' '2023Q1' '2024Q2' '2020Q4' '2021Q2'
 '2021Q3' '2021Q1' '2021Q4' '2022Q1' '2022Q2' '2022Q3' '2022Q4' '2024Q3'
 '2024Q4' '2025Q1' '2025Q2']
Quarters in cpi_index: ['2021Q1' '2021Q2' '2021Q3' '2021Q4' '2022Q1' '2022Q2' '2022Q3' '2022Q4'
 '2023Q1' '2023Q2' '2023Q3' '2023Q4' '2024Q1' '2024Q2' '2024Q3' '2024Q4'
 '2025Q1']


## Data Cleaning
1. Remove Columns that will not be used 
2. Remove duplicate Rows
3. Remove false negatives

In [14]:
# Step 1: Drop unnecessary columns
columns_to_drop = ['FCID', 'BCreatedDateAEST', 'CPI', 'Quarter', 'PriceIncGSTRaw' , 'VRego', 'BShopID', 'BShopRegionName', 'BShopPostcode', 'IsDeleted', 'BStatusFromDateTimeAEST', 'Date', 'BStatusFinal', ]
df.drop(columns = columns_to_drop, inplace=True)
df.head()

,BookingID,BStatusAfterSubmitted,BTicketID,BTicketType,TaskName,IsCustomService,IsCustomRepair,VYMM,VMakeModel,VMake,VYear,VMid,BShopState,BShopRegionClass,Distance,Months,AdjustedPrice
0,1449822,33. Approved,2323471,Log,Logbook Service,0,0,2019 AUDI A5 COUPE,AUDI A5 COUPE,AUDI,2019.0,AUD45780,ACT,2.0,75000.0,60.0,475.0
1,1565721,33. Approved,2537851,Capped,Capped Price Service,0,0,2022 TOYOTA HILUX,TOYOTA HILUX,TOYOTA,2022.0,TOY47062,WA,2.0,10000.0,NaN,297.0
2,1448252,33. Approved,2320054,Capped,Capped Price Service,0,0,2019 HYUNDAI SANTA FE,HYUNDAI SANTA FE,HYUNDAI,2019.0,HYU45824,VIC,2.0,105000.0,NaN,367.0
3,1320517,33. Approved,2051287,Capped,Capped Price Service,0,0,2021 TOYOTA RAV4,TOYOTA RAV4,TOYOTA,2021.0,TOY45523,NSW,3.0,75000.0,NaN,238.0
4,1551817,16. Requires Changes,2512709,Repair,Roadworthy Inspection / Pink Slips,0,0,2015 HYUNDAI I30,HYUNDAI I30,HYUNDAI,2015.0,HYU38277,NSW,2.0,NaN,NaN,47.0


In [15]:
# Step 2: Remove Duplicate Rows
# Duplicates can skew analysis and lead to incorrect conclusions, thus it is important each row is unique. Duplicates in this dataset are considered as rows that contain the same values across all columns other than the ID columns 
print(f"size before: {df.shape}" )
df = df.drop_duplicates(subset=[col for col in df.columns if col not in ['BookingID', 'BTicketID']])
print(f"size after: {df.shape}" )

size before: (876326, 17)
size after: (697584, 17)


In [16]:
# Step 3: Remove false negatives
# False negatives are rows which have a status of '16. Requires Changes' when they should have a status of '33. Approved'. This occurs in the dataset as bookings contain multiple tickets. If a single ticket in a booking requires changes, the entire booking is marked as 'Requires Changes', thus marking tickets that do not require changes incorrectly. These false negatives can be detected and removed by checking if a duplicate entry exists where only the status changes.
print(f"size before: {df.shape}" )
df['StatusPriority'] = df['BStatusAfterSubmitted'].apply(lambda x: 0 if x == '33. Approved' else 1) # Assign priority: approved gets highest priority (lowest number)
dedup_cols = [col for col in df.columns if col not in ['BStatusAfterSubmitted', 'StatusPriority']] # Define columns to check for duplicates
df = df.sort_values(by=dedup_cols + ['StatusPriority']) # Sort so approved status is first
df = df.drop_duplicates(subset=dedup_cols, keep='first') # Keep the first occurrence (which is the approved status)
df = df.drop(columns='StatusPriority') # drop the temporary column used for sorting


df['Label'] = df['BStatusAfterSubmitted'].map({'33. Approved': 1, '16. Requires Changes': 0, '29. Rejected': 0}) # create label column for model training
df = df.drop(columns=['BStatusAfterSubmitted'], axis=1) # drop the original status column as it is no longer needed
print(f"size after: {df.shape}" ) 

size before: (697584, 17)
size after: (605686, 17)


In [17]:
# Step 4: Drop final columns
columns_to_drop = ['BookingID', 'BTicketID', 'IsCustomService', 'IsCustomRepair', 'VYMM', 'BShopRegionClass', 'BShopState', 'VMid', 'BTicketType']
df = df.drop(columns=columns_to_drop, axis=1)

In [18]:
df = df[df['TaskName'] == 'Logbook Service'] 

In [19]:
df.head()

,TaskName,VMakeModel,VMake,VYear,Distance,Months,AdjustedPrice,Label
42112,Logbook Service,TOYOTA RAV4,TOYOTA,2016.0,320000.0,192.0,370.0,1
37740,Logbook Service,MAZDA 3,MAZDA,2015.0,160000.0,192.0,454.0,1
37761,Logbook Service,TOYOTA HIACE,TOYOTA,2010.0,260000.0,156.0,367.0,1
45678,Logbook Service,SUBARU OUTBACK,SUBARU,2018.0,50000.0,24.0,568.0,1
39797,Logbook Service,TOYOTA HIACE,TOYOTA,2015.0,160000.0,96.0,939.0,1


In [20]:
df = df.drop(columns=['TaskName'], axis=1)

In [21]:
print(f"size before dropping missing values: {df.shape}")
#df.dropna(inplace=True) # drop rows with missing values
print(f"size after dropping missing values: {df.shape}")

size before dropping missing values: (163067, 7)
size after dropping missing values: (163067, 7)


In [22]:
df.to_csv('data/preprocessed_data.csv', index=False)